# Setting Dependencies

In [ ]:
import csv
import os
import random
import shutil
import string
from pathlib import Path
import cv2
import uuid
import numpy as np
import pandas
import tensorflow as tsf
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_recall_fscore_support, confusion_matrix

from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import (Add, Activation, AveragePooling2D, BatchNormalization, Concatenate, Conv2D,
                          Dense, Dropout, Flatten, GlobalAveragePooling2D, Input, MaxPooling2D)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from google.colab import drive
from tensorflow import keras

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import tensorflow as tf
import keras
print("Python version:", sys.version)
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
TensorFlow version: 2.15.0
Keras version: 2.15.0


# Data Utilities for image preparation

This is the base model class which all models inherit from. It is a empty sequential model containing multiple important methods for training and input managing

In [ ]:
@keras.utils.register_keras_serializable(package='Custom', name='Model_Base')
class Model_Base(Sequential):
  def __init__(self, *args, **kwargs):
    super(Model_Base, self).__init__(*args, **kwargs)

  def callbackList(self, logPath):
    early_stopping = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True)
    LR = ReduceLROnPlateau(monitor = "val_loss",
                           factor=0.05,
                           patience=5,
                           min_lr=0.0001)
    if not (logPath == "agelogs" or logPath == "genderlogs"):
        raise ValueError("logPath value must be agelogs or genderlogs")
    else:
        log_path = os.path.join("/content/drive/My Drive/models",  logPath)
        model_checkpoint_name = input("Enter save name for the model: ")
        checkpoint_path = os.path.join(log_path, model_checkpoint_name)

        checkpointer = ModelCheckpoint(filepath=checkpoint_path, monitor='loss',verbose=1,save_best_only=True,
                           save_weights_only=False, mode='auto',save_freq='epoch')

        callback_list=[checkpointer, early_stopping]
        return list(callback_list), checkpoint_path

  def preprocess_images(self, image_files):
      images = []
      for file in image_files:
          img = cv2.imread(file)
          if img is not None:
              img = cv2.resize(img, (224, 224))
              img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
              img = img / 255.0
              images.append(img)
              print(file)
          else:
              print(f"Failed to load image: {file}")
      return np.array(images)

  #Assigns random names to each model to prevent any layer name repeat issues when loading the model
  def layer_name_random(label = "modelLayer"):
    uniquePrefix = int(uuid.uuid4().int)
    uniquePrefix = str(uniquePrefix)
    uniquePrefix = uniquePrefix[:10]
    return uniquePrefix




# - GENDER FUNCTIONS


  def gender_inputs(self):
    print("TRAINING")
    training_csv = pandas.read_csv(Path("/content/drive/My Drive/models/TrainingRGB.csv"))
    print("VALIDATION")
    testing_csv = pandas.read_csv(Path("/content/drive/My Drive/models/ValidationRGB.csv"))

    print("Setting gender target")

    X_train = training_csv["Name"].values
    y_train = training_csv["gender"].values

    X_test = testing_csv["Name"].values
    y_test = testing_csv["gender"].values

    X_train = self.preprocess_images(X_train)
    X_test = self.preprocess_images(X_test)

    return X_train, X_test, y_train, y_test


  def gender_training(self, batch_size=80):
    print("inputs")
    X_train, X_test, y_train, y_test = self.gender_inputs()

    print("Callback list")
    callback_list, log_path = self.callbackList("genderlogs")


    history = self.fit(X_train, y_train, batch_size=batch_size , validation_batch_size=batch_size,
                           validation_data=(X_test, y_test),epochs=60,
                           callbacks=callback_list,shuffle=True)
    prediction = self.predict(X_test)

    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)

    prediction = (prediction > 0.5).astype(int)

    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)

    acc = accuracy_score(y_test, prediction)
    prec, recall, f1, support = precision_recall_fscore_support(y_test, prediction, average='macro')
    confusionMatrix = confusion_matrix(y_test, prediction)

    print(f"Accuracy is {acc}")
    print(f"Precision is:  {prec}")
    print(f"Recall is:  {recall}")
    print(f"F1 is :  {f1}")
    print(confusionMatrix)
    return history


# - Age classification  FUNCTIONS
  def label_classes(self, age):
    age = int(age)
    if 1<=age<=3:
        return 0
    elif 4<=age<=9:
        return 1
    elif 10<=age<=19:
        return 2
    elif 20<=age<=35:
        return 3
    elif 36<=age<=45:
        return 4
    elif 46<=age<=60:
        return 5
    else:
        return 6


  def age_inputs(self):
    training_csv = pandas.read_csv(Path("/content/drive/My Drive/models/TrainingRGB.csv"))
    testing_csv = pandas.read_csv(Path("/content/drive/My Drive/models/ValidationRGB.csv"))

    train_files = training_csv["Name"].values
    train_labels = training_csv["age"].map(self.label_classes)

    test_files = testing_csv["Name"].values
    test_labels = testing_csv["age"].map(self.label_classes)
    print(test_labels)
    del training_csv, testing_csv

    print("Preprocessing Images")
    print("Train csv")
    X_train = self.preprocess_images(train_files)
    print("test csv")
    X_test = self.preprocess_images(test_files)

    y_train = train_labels
    y_test = test_labels

    return X_train, X_test, y_train, y_test

  def age_training(self, batch_size=80):
    X_train, X_test, y_train, y_test = self.age_inputs()
    print("Callback list")
    callback_list, log_path = self.callbackList("agelogs")
    print("Compile")

    print("Fit")
    history = self.fit(X_train, y_train, batch_size=batch_size, validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=60,
                            callbacks=callback_list,shuffle=True)


    #prediction = model.predict(X_test)
    prediction = self.predict(X_test)
    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)
    prediction= np.argmax(prediction, axis=1)
    acc = accuracy_score(y_test, prediction)
    prec, recall, f1, support = precision_recall_fscore_support(y_test, prediction, average='macro')
    confusionMatrix = confusion_matrix(y_test, prediction)

    print(f"Accuracy is {acc}")
    print(f"Precision is:  {prec}")
    print(f"Recall is:  {recall}")
    print(f"F1 is :  {f1}")
    print(confusionMatrix)
    return history

#Age regression functions
  def regression_inputs(self):
    training_csv = pandas.read_csv(Path("/content/drive/My Drive/models/TrainingRGB.csv"))
    testing_csv = pandas.read_csv(Path("/content/drive/My Drive/models/ValidationRGB.csv"))

    train_files = training_csv["Name"].values
    train_labels = training_csv["age"]

    test_files = testing_csv["Name"].values
    test_labels = testing_csv["age"]
    print(test_labels)
    del training_csv, testing_csv

    print("Preprocessing Images")
    print("Train csv")
    X_train = self.preprocess_images(train_files)
    print("test csv")
    X_test = self.preprocess_images(test_files)

    y_train = train_labels
    y_test = test_labels

    return X_train, X_test, y_train, y_test

  def regression_training(self, batch_size=80):
    X_train, X_test, y_train, y_test = self.regression_inputs()
    print("Callback list")
    callback_list, log_path = self.callbackList("agelogs")
    print("Compile")

    print("Fit")
    history = self.fit(X_train, y_train, batch_size=batch_size, validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=60,
                            callbacks=callback_list,shuffle=True)


    #prediction = model.predict(X_test)
    prediction = self.predict(X_test)
    rSquared = r2_score(y_test, prediction.flatten())
    meanAbsoluteError = mean_absolute_error(y_test, prediction.flatten())
    meanSquaredError = mean_squared_error(y_test, prediction.flatten())
    rootMeanSquaredError = mean_squared_error(y_test, prediction.flatten(), squared=False)
    print(f"R-Squared value is {rSquared}")
    print(f"Mean absolute error is {meanAbsoluteError}")
    print(f"Mean squared error is {meanSquaredError}")
    print(f"Root mean squared error is {rootMeanSquaredError}")
    return history


#Model layers
#This section 


  #Identity  block for ResNet18 and 34
  def identity_block(self, input_tensor, filters):

    identBlock = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding='same')(input_tensor)
    identBlock = BatchNormalization(axis=-1)(identBlock)
    identBlock = Activation("relu")(identBlock)
    identBlock = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding='same')(identBlock)
    identBlock = BatchNormalization(axis=-1)(identBlock)


    outputTensor = Add()([input_tensor, identBlock])
    outputTensor = Activation("relu")(outputTensor)
    return outputTensor


  #Convolutional block for ResNet18 and 34
  def residual_block(self, input_tensor, filters):

    shortcutConnection = Conv2D(filters=filters, kernel_size=(1, 1), strides=(2,2), padding='same')(input_tensor)
    shortcutConnection = BatchNormalization(axis=-1)(shortcutConnection)

    convBlock = Conv2D(filters=filters, kernel_size=(1, 1), strides=(2,2), padding='same')(input_tensor)
    convBlock = BatchNormalization(axis=-1)(convBlock)
    convBlock = Activation("relu")(convBlock)

    convBlock = Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), padding='same')(convBlock)
    convBlock = BatchNormalization(axis=-1)(convBlock)
    convBlock = Activation("relu")(convBlock)

    convBlock = Conv2D(filters=filters, kernel_size=(1, 1), strides=(1, 1), padding='same')(convBlock)
    convBlock = BatchNormalization(axis=-1)(convBlock)

    output_tensor = Add()([convBlock, shortcutConnection])
    output_tensor = Activation("relu")(output_tensor)

    return output_tensor





# Identity and Dense block

In [ ]:
class ResNet_misc:

    #This function defines the identity block or skip connections. This allows the model to learn through a residual function
    def identity_block(input_tensor, filters, strides = (1,1)):
        identBlock = Conv2D(filters=filters, kernel_size=(3,3), strides=strides, padding='same', kernel_regularizer=l2(0.01))(input_tensor)#
        identBlock = BatchNormalization(axis=-1)(identBlock)
        identBlock = Activation("relu")(identBlock)
        identBlock = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.01))(identBlock)#
        identBlock = BatchNormalization(axis=-1)(identBlock)

        if strides != (1, 1):
            input_tensor = Conv2D(filters=filters, kernel_size=(1,1), strides=strides, padding='same', kernel_regularizer=l2(0.01))(input_tensor)
            input_tensor = BatchNormalization()(input_tensor)


        outputTensor = Add()([input_tensor, identBlock])
        outputTensor = Activation("relu")(outputTensor)
        return outputTensor

    def dense_block(input_tensor, filters, dropout_rate, layer_num=7):
        compression = 0.5

        output_tensor = input_tensor

        for layer in range(layer_num):
            x = BatchNormalization()(output_tensor)
            x = Activation('relu')(x)
            x = Conv2D(int(filters*compression), (3,3), use_bias=False ,padding='same', kernel_regularizer=l2(0.01))(x)
            x = Dropout(dropout_rate)(x)
            output_tensor = Concatenate(axis=-1)([output_tensor,x])

        return output_tensor

# Simple Gender Model

In [ ]:
@keras.utils.register_keras_serializable(package='Custom', name='Gender_Basic')
class Gender_Basic(Model_Base):
  def __init__(self, *args, **kwargs):
    super(Gender_Basic, self).__init__(*args, **kwargs)

    self.add(Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001), name=self.layer_name_random()))
    self.add(Dropout(0.1, name=self.layer_name_random()))
    self.add(Activation('relu', name=self.layer_name_random()))
    self.add(MaxPooling2D(pool_size = (2,2), name=self.layer_name_random()))

    self.add(Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001), name=self.layer_name_random()))
    self.add(Dropout(0.1, name=self.layer_name_random()))
    self.add(Activation('relu', name=self.layer_name_random()))
    self.add(MaxPooling2D(pool_size = (2,2), name=self.layer_name_random()))

    self.add(Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001), name=self.layer_name_random()))
    self.add(Dropout(0.1, name=self.layer_name_random()))
    self.add(Activation('relu', name=self.layer_name_random()))
    self.add(MaxPooling2D(pool_size = (2,2), name=self.layer_name_random()))

    self.add(Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001), name=self.layer_name_random()))
    self.add(Dropout(0.1, name=self.layer_name_random()))
    self.add(MaxPooling2D(pool_size = (2,2), name=self.layer_name_random()))

    self.add(Flatten(name=self.layer_name_random()))
    self.add(Dense(128,activation='relu', name=self.layer_name_random()))
    self.add(Dropout(0.2, name=self.layer_name_random()))
    self.add(Dense(1,activation="sigmoid", name=self.layer_name_random()))
    self.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])



model = Gender_Basic()
history = model.gender_training()


"""
As there is only one way to call this method/architecture
the only thing required was to set an instance of the class and then call its Gender_training
method. Other models with multiple target options, multiple different layer amounts and a regression
option require additional methods outside of the class that allow the person running the model to
control what specific model is being trained using the terminal
"""




# Simple Age Model

In [ ]:
@keras.utils.register_keras_serializable(package='Custom', name='Age_Base')
class Age_Base(Model_Base):
  def __init__(self, *args, **kwargs):
    super(Age_Base, self).__init__(*args, **kwargs)
    self.add(Conv2D(32, (3,3), activation='relu'))
    self.add(AveragePooling2D(pool_size=(2,2)))
    self.add(Dropout(0.15))

    self.add(Conv2D(64, (3,3), activation='relu'))
    self.add(AveragePooling2D(pool_size=(2,2)))

    self.add(Conv2D(128, (3,3), activation='relu'))
    self.add(AveragePooling2D(pool_size=(2,2)))
    self.add(Dropout(0.15))

    self.add(Conv2D(128, (3,3), activation='relu'))
    self.add(AveragePooling2D(pool_size=(2,2)))
    self.add(Dropout(0.15))

    self.add(Conv2D(128, (3,3), activation='relu'))
    self.add(AveragePooling2D(pool_size=(2,2)))
    self.add(Dropout(0.15))

    self.add(Conv2D(256, (3,3), activation='relu'))
    self.add(AveragePooling2D(pool_size=(2,2)))

    self.add(GlobalAveragePooling2D())
    self.add(Dense(132, activation='relu'))
    self.add(Dropout(0.15))

@keras.utils.register_keras_serializable(package='Custom', name='Age_Classification')
class Age_Classification(Age_Base):
  def __init__(self, *args, **kwargs):
    super(Age_Classification, self).__init__(*args, **kwargs)
    self.add(Dense(7, activation='softmax'))
    self.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='Age_Regression')
class Age_Regression(Age_Base):
  def __init__(self, *args, **kwargs):
    super(Age_Regression, self).__init__(*args, **kwargs)
    self.add(Dense(1, activation='linear'))
    self.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])


def ageModelBuilding():
  selection = input("Select either [(1=Classification)(2=Regression)]")
  if selection == "1":
    model = Age_Classification()
    history = model.age_training()
  elif selection == "2":
    model = Age_Regression()
    history = model.regression_training()
  else:
    print("Input value error. Only input either 1 or 2")
    ageModelBuilding()

ageModelBuilding()

# VGG-16/19

In [ ]:
#VGG like Architecture but with only 7 layers
@keras.utils.register_keras_serializable(package='Custom', name='VGGsimple')
class VGGsimple(Model_Base):
  def __init__(self, *args, **kwargs):
    super(VGGsimple, self).__init__(*args, **kwargs)
    self.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same", input_shape=(224, 224, 3)))
    self.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"))
    self.add(MaxPooling2D(pool_size=(2, 2)))

    self.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"))
    self.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"))
    self.add(MaxPooling2D(pool_size=(2, 2)))

    self.add(Conv2D(256, kernel_size=(3, 3), activation="relu", padding="same"))
    self.add(Conv2D(256, kernel_size=(3, 3), activation="relu", padding="same"))
    self.add(Conv2D(256, kernel_size=(3, 3), activation="relu", padding="same"))
    self.add(MaxPooling2D(pool_size=(2, 2)))

    self.add(Flatten())
    self.add(Dense(512, activation='relu'))
    self.add(Dropout(0.5))

@keras.utils.register_keras_serializable(package='Custom', name='VGGsimpleGENDER')
class VGGsimpleGENDER(VGGsimple):
  def __init__(self, *args, **kwargs):
    super(VGGsimpleGENDER, self).__init__()
    self.add(Dense(1, activation='sigmoid'))
    self.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='VGGsimpleAGE')
class VGGsimpleAGE(VGGsimple):
  def __init__(self, *args, **kwargs):
    super(VGGsimpleAGE, self).__init__(*args, **kwargs)
    self.add(Dense(7, activation='softmax'))
    self.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam', metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='VGGsimpleREGRESSION')
class VGGsimpleREGRESSION(VGGsimple):
  def __init__(self, *args, **kwargs):
    super(VGGsimpleREGRESSION, self).__init__(*args, **kwargs)
    self.add(Dense(1, activation='linear'))
    self.compile(loss='mean_squared_error', optimizer='adam',
                 metrics=['mae'])





In [ ]:
#VGG-16 architecture
@keras.utils.register_keras_serializable(package='Custom', name='VGG')
class VGG(Model_Base):
  def __init__(self, model19=False, *args, **kwargs):
    super(VGG, self).__init__(model19, *args, **kwargs)
    self.add(Conv2D(64, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(64, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Conv2D(128, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(128, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Conv2D(256, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(256, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(256, kernel_size=(3,3), activation= "relu", padding="same"))
    #if model19:
    self.add(Conv2D(256, kernel_size=(3,3), activation= "relu", padding="same"))

    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    #if model19:
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))

    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    #if model19:
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))

    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Flatten())
    self.add(Dense(4096, activation='relu'))

@keras.utils.register_keras_serializable(package='Custom', name='VGG_GENDER')
class VGG_GENDER(VGG):
  def __init__(self, model19=False, *args, **kwargs):
     super(VGG_GENDER, self).__init__(model19=model19, *args, **kwargs)
     self.add(Dense(1, activation= 'sigmoid'))
     self.compile(optimizer= "adam",
                  loss="binary_crossentropy",
                  metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='VGG_AGE')
class VGG_AGE(VGG):
  def __init__(self, model19=False, *args, **kwargs):
     super(VGG_AGE, self).__init__(model19=model19, *args, **kwargs)
     self.add(Dense(7, activation= 'softmax'))
     self.compile(optimizer= "adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])


@keras.utils.register_keras_serializable(package='Custom', name='VGG_Regression')
class VGG_Regression(VGG):
  def __init__(self, model19=False, *args, **kwargs):
    super(VGG_Regression, self).__init__(model19=model19, *args, **kwargs)
    self.add(Dense(1, activation= 'linear'))
    self.compile(optimizer= "adam",
                 loss="mean_squared_error",
                 metrics=['mae'])






def selectVGGmodel():
  select = input("Please input corresponding number to the model you want to train [(1=Gender)(2=Age)(3=Age regression)]")
  if select == "1":
    print("gender")
    #GENDER
    model = VGG_GENDER()
    history = model.gender_training()
  elif select=="2":
    #AGE
    model = VGG_AGE()
    history = model.age_training()
  elif select == "3":
    #REGRESSION
    model = VGG_Regression()
    history = model.regression_training()
  else:
    raise ValueError("Please input either 1, 2 or 3")

selectVGGmodel()


In [ ]:
#VGG19 architecture
@keras.utils.register_keras_serializable(package='Custom', name='VGG19')
class VGG19(Model_Base):
  def __init__(self, model19=False, *args, **kwargs):
    super(VGG19, self).__init__(model19, *args, **kwargs)
    self.add(Conv2D(64, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(64, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Conv2D(128, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(128, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Conv2D(256, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(256, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(256, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(256, kernel_size=(3,3), activation= "relu", padding="same"))

    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))

    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))
    self.add(Conv2D(512, kernel_size=(3,3), activation= "relu", padding="same"))

    self.add(MaxPooling2D(pool_size=(2,2)))

    self.add(Flatten())
    self.add(Dense(4096, activation='relu'))

@keras.utils.register_keras_serializable(package='Custom', name='VGG19_GENDER')
class VGG19_GENDER(VGG19):
  def __init__(self, model19=False, *args, **kwargs):
     super(VGG19_GENDER, self).__init__(model19=model19, *args, **kwargs)
     self.add(Dense(1, activation= 'sigmoid'))
     self.compile(optimizer= "adam",
                  loss="binary_crossentropy",
                  metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='VGG19_AGE')
class VGG19_AGE(VGG19):
  def __init__(self, model19=False, *args, **kwargs):
     super(VGG_AGE, self).__init__(model19=model19, *args, **kwargs)
     self.add(Dense(7, activation= 'softmax'))
     self.compile(optimizer= "adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])


@keras.utils.register_keras_serializable(package='Custom', name='VGG19_Regression')
class VGG19_Regression(VGG19):
  def __init__(self, model19=False, *args, **kwargs):
    super(VGG_Regression, self).__init__(model19=model19, *args, **kwargs)
    self.add(Dense(1, activation= 'linear'))
    self.compile(optimizer= "adam",
                 loss="mean_squared_error",
                 metrics=['mae'])






def selectVGGmodel():
  select = input("Please input corresponding number to the model you want to train [(1=Gender)(2=Age)(3=Age regression)]")
  if select == "1":
    print("gender")
    #GENDER
    model = VGG19_GENDER()
    history = model.gender_training()
  elif select=="2":
    #AGE
    model = VGG19_AGE()
    history = model.age_training()
  elif select == "3":
    #REGRESSION
    model = VGG19_Regression()
    history = model.regression_training()
  else:
    raise ValueError("Please input either 1, 2 or 3")

selectVGGmodel()

# ResNet-18/34

In [ ]:
@keras.utils.register_keras_serializable(package='Custom', name='ResNet18')
class ResNet18(Model_Base):
  def __init__(self, layers34=False, *args, **kwargs):
    super(ResNet18, self).__init__( *args, **kwargs)
    self.build()
  #This method is a solutio
  def build(self):
    self.inputs = Input((224, 224, 3))

    # Conv1
    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same', kernel_regularizer=l2(0.01))(self.inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # conv2_x
    x = self.identity_block(x, filters=64)
    x = self.identity_block(x, filters=64)

    # conv3_x
    x = self.residual_block(x, filters=128)
    x = self.identity_block(x, filters=128)

    # conv4_x
    x = self.residual_block(x, filters=256)
    x = self.identity_block(x, filters=256)

    # conv5_x
    x = self.residual_block(x, filters=512)
    x = self.identity_block(x, filters=512)
    if self.layers34:
      x = self.identity_block(x, filters=512)

    x = GlobalAveragePooling2D()(x)

    self.baseLayers = x


#A quirk in keras.sequental prevents any skip connections requiring a slightly different training function

  def gender_training(self, batch_size=80):
    print("inputs")
    X_train, X_test, y_train, y_test = self.gender_inputs()

    print("Callback list")
    callback_list, log_path = self.callbackList("genderlogs")


    history = self.fit(X_train, y_train, batch_size=batch_size , validation_batch_size=batch_size,
                           validation_data=(X_test, y_test),epochs=60,
                           callbacks=callback_list,shuffle=True)
    prediction = self.predict(X_test)

    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)

    prediction = (prediction > 0.5).astype(int)

    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)

    acc = accuracy_score(y_test, prediction)
    prec, recall, f1, support = precision_recall_fscore_support(y_test, prediction, average='macro')
    confusionMatrix = confusion_matrix(y_test, prediction)

    print(f"Accuracy is {acc}")
    print(f"Precision is:  {prec}")
    print(f"Recall is:  {recall}")
    print(f"F1 is :  {f1}")
    print(confusionMatrix)
    return history


  def regression_training(self, batch_size=80):
    X_train, X_test, y_train, y_test = self.regression_inputs()
    print("Callback list")
    callback_list, log_path = self.callbackList("agelogs")
    print("Compile")

    print("Fit")
    history = self.model.fit(X_train, y_train, batch_size=batch_size, validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=30,
                            callbacks=callback_list,shuffle=True)


    prediction = self.model.predict(X_test)
    rSquared = r2_score(y_test, prediction.flatten())
    meanAbsoluteError = mean_absolute_error(y_test, prediction.flatten())
    meanSquaredError = mean_squared_error(y_test, prediction.flatten())
    rootMeanSquaredError = mean_squared_error(y_test, prediction.flatten(), squared=False)
    print(f"R-Squared value is {rSquared}")
    print(f"Mean absolute error is {meanAbsoluteError}")
    print(f"Mean squared error is {meanSquaredError}")
    print(f"Root mean squared error is {rootMeanSquaredError}")
    return history


  def age_training(self, batch_size=80):
    X_train, X_test, y_train, y_test = self.age_inputs()
    print("Callback list")
    callback_list, log_path = self.callbackList("agelogs")
    print("Compile")

    print("Fit")
    history = self.model.fit(X_train, y_train, batch_size=batch_size, validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=30,
                            callbacks=callback_list,shuffle=True)


    prediction = self.model.predict(X_test)
    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)
    prediction= np.argmax(prediction, axis=1)
    acc = accuracy_score(y_test, prediction)
    prec, recall, f1, support = precision_recall_fscore_support(y_test, prediction, average='macro')
    confusionMatrix = confusion_matrix(y_test, prediction)
    print(f"Accuracy is {acc}")
    print(f"Precision is:  {prec}")
    print(f"Recall is:  {recall}")
    print(f"F1 is :  {f1}")
    print(confusionMatrix)
    return history








@keras.utils.register_keras_serializable(package='Custom', name='ResNetGender')
class ResNetGender(ResNet18):
  def __init__(self, *args, **kwargs):
    super(ResNetGender, self).__init__(*args, **kwargs)
    outputs = Dense(1, activation='sigmoid')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='ResNetAge')
class ResNetAge(ResNet18):
  def __init__(self, *args, **kwargs):
    super(ResNetAge, self).__init__( *args, **kwargs)
    outputs = Dense(7, activation='softmax')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])


@keras.utils.register_keras_serializable(package='Custom', name='ResNetRegression')
class ResNetRegression(ResNet18):
  def __init__(self, *args, **kwargs):
    super(ResNetRegression, self).__init__( *args, **kwargs)
    outputs = Dense(1, activation='linear')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(loss='mean_squared_error', optimizer='adam',
                 metrics=['mae'])

resmod = ResNetRegression()
history = resmod.age_training()

KeyboardInterrupt: 

In [ ]:
@keras.utils.register_keras_serializable(package='Custom', name='ResNet34')
class ResNet34(Model_Base):
  def __init__(self, layers34=False, *args, **kwargs):
    super(ResNet34, self).__init__( *args, **kwargs)
    self.build()

  #Sets thearchitecure of the model. It was made this way to try to get arround an unusual bug with the sequential class which wont allow skip connections
  #Very similar architecture to ResNet but with increased number of idenity blocks
  def build(self):
    self.inputs = Input((224, 224, 3))

    # Conv1
    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same', kernel_regularizer=l2(0.01))(self.inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # conv2_x
    x = self.identity_block(x, filters=64)
    x = self.identity_block(x, filters=64)
    x = self.identity_block(x, filters=64)

    # conv3_x
    x = self.residual_block(x, filters=128)
    x = self.identity_block(x, filters=128)
    x = self.identity_block(x, filters=128)
    x = self.identity_block(x, filters=128)

    # conv4_x
    x = self.residual_block(x, filters=256)
    x = self.identity_block(x, filters=256)
    x = self.identity_block(x, filters=256)
    x = self.identity_block(x, filters=256)
    x = self.identity_block(x, filters=256)
    x = self.identity_block(x, filters=256)

    # conv5_x
    x = self.residual_block(x, filters=512)
    x = self.identity_block(x, filters=512)
    x = self.identity_block(x, filters=512)

    x = GlobalAveragePooling2D()(x)

    self.baseLayers = x


#A quirk in keras.sequental prevents any skip connections requiring a slightly different training function

  def gender_training(self, batch_size=80):
    print("inputs")
    X_train, X_test, y_train, y_test = self.gender_inputs()

    print("Callback list")
    callback_list, log_path = self.callbackList("genderlogs")


    history = self.model.fit(X_train, y_train, batch_size=batch_size , validation_batch_size=batch_size,
                           validation_data=(X_test, y_test),epochs=60,
                           callbacks=callback_list,shuffle=True)
    prediction = self.model.predict(X_test)

    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)

    prediction = (prediction > 0.5).astype(int)

    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)

    acc = accuracy_score(y_test, prediction)
    prec, recall, f1, support = precision_recall_fscore_support(y_test, prediction, average='macro')
    confusionMatrix = confusion_matrix(y_test, prediction)

    print(f"Accuracy is {acc}")
    print(f"Precision is:  {prec}")
    print(f"Recall is:  {recall}")
    print(f"F1 is :  {f1}")
    print(confusionMatrix)
    return history


  def regression_training(self, batch_size=80):
    X_train, X_test, y_train, y_test = self.regression_inputs()
    print("Callback list")
    callback_list, log_path = self.callbackList("agelogs")
    print("Compile")

    print("Fit")
    history = self.model.fit(X_train, y_train, batch_size=batch_size, validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=60,
                            callbacks=callback_list,shuffle=True)


    prediction = self.model.predict(X_test)
    rSquared = r2_score(y_test, prediction.flatten())
    meanAbsoluteError = mean_absolute_error(y_test, prediction.flatten())
    meanSquaredError = mean_squared_error(y_test, prediction.flatten())
    rootMeanSquaredError = mean_squared_error(y_test, prediction.flatten(), squared=False)
    print(f"R-Squared value is {rSquared}")
    print(f"Mean absolute error is {meanAbsoluteError}")
    print(f"Mean squared error is {meanSquaredError}")
    print(f"Root mean squared error is {rootMeanSquaredError}")
    return history


  def age_training(self, batch_size=80):
    X_train, X_test, y_train, y_test = self.age_inputs()
    print("Callback list")
    callback_list, log_path = self.callbackList("agelogs")
    print("Compile")

    print("Fit")
    history = self.model.fit(X_train, y_train, batch_size=batch_size, validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=60,
                            callbacks=callback_list,shuffle=True)


    prediction = self.model.predict(X_test)
    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)
    prediction= np.argmax(prediction, axis=1)
    acc = accuracy_score(y_test, prediction)
    prec, recall, f1, support = precision_recall_fscore_support(y_test, prediction, average='macro')
    confusionMatrix = confusion_matrix(y_test, prediction)
    print(f"Accuracy is {acc}")
    print(f"Precision is:  {prec}")
    print(f"Recall is:  {recall}")
    print(f"F1 is :  {f1}")
    print(confusionMatrix)
    return history






# Model for building the specific versions of ResNet-34

@keras.utils.register_keras_serializable(package='Custom', name='ResNetGender34')
class ResNetGender34(ResNet34):
  def __init__(self, *args, **kwargs):
    super(ResNetGender34, self).__init__(*args, **kwargs)
    outputs = Dense(1, activation='sigmoid')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='ResNetAge34')
class ResNetAge34(ResNet34):
  def __init__(self, *args, **kwargs):
    super(ResNetAge34, self).__init__( *args, **kwargs)
    outputs = Dense(7, activation='softmax')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])


@keras.utils.register_keras_serializable(package='Custom', name='ResNetRegression34')
class ResNetRegression34(ResNet34):
  def __init__(self, *args, **kwargs):
    super(ResNetRegression34, self).__init__( *args, **kwargs)
    outputs = Dense(1, activation='linear')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(loss='mean_squared_error', optimizer='adam',
                 metrics=['mae'])

resmod = ResNetRegression34()
history = resmod.regression_training()

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/ValidationData/57_0_0_20170105173447996.jpg.chip.jpg
/content/drive/My Drive/ValidationData/61_1_3_20170105001450436.jpg.chip.jpg
/content/drive/My Drive/ValidationData/45_1_1_20170112203850233.jpg.chip.jpg
/content/drive/My Drive/ValidationData/31_1_1_20170112211518637.jpg.chip.jpg
/content/drive/My Drive/ValidationData/26_1_3_20170119193142729.jpg.chip.jpg
/content/drive/My Drive/ValidationData/40_1_0_20170117150751092.jpg.chip.jpg
/content/drive/My Drive/ValidationData/37_1_3_20170119202521964.jpg.chip.jpg
/content/drive/My Drive/ValidationData/50_0_3_20170119205538583.jpg.chip.jpg
/content/drive/My Drive/ValidationData/32_1_1_20170112210316260.jpg.chip.jpg
/content/drive/My Drive/ValidationData/60_1_1_20170113011821480.jpg.chip.jpg
/content/drive/My Drive/ValidationData/2_0_0_20170104201100490.jpg.chip.jpg
/content/drive/My Drive/ValidationData/27_1_3_20170104231741609.jpg.chip.jpg
/content/drive/My Drive/Va

# DenseNet

In [ ]:
@keras.utils.register_keras_serializable(package='Custom', name='DenseNet')
class DenseNet(Model_Base):
  def __init__(self, *args, **kwargs):
    super(DenseNet, self).__init__(*args, **kwargs)
    self.compression = 0.5
    self.inputs = Input((224, 224, 3))
    x = Conv2D(30, (3,3), use_bias=False, padding='same')(self.inputs)
    x = self.dense_block(x, 30)
    # Transition between denseblocks
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(12, (1,1), use_bias=False, padding='same', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.2)(x)
    x = AveragePooling2D(pool_size=(2,2))(x)
    #  over
    x = self.dense_block(x, 60)
    #  output
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    self.baseLayers = x



  #This is the dense block which adds all previous output to the inputs of each layer
  def dense_block(self, input_tensor, filters):
    output_tensor = input_tensor
    for layer in range(2):
      x = BatchNormalization()(output_tensor)
      x = Activation('relu')(x)
      x = Conv2D(filters, (3,3), use_bias=False, padding='same', kernel_regularizer=l2(0.01))(x)
      x = Dropout(0.5)(x)
      output_tensor = Concatenate(axis=-1)([output_tensor, x])
    return output_tensor


#A quirk in keras.sequental prevents any skip connections requiring a slightly different training function. Using OOP polymorphism to allow for this change
  def gender_training(self, batch_size=80):
    print("inputs")
    X_train, X_test, y_train, y_test = self.gender_inputs()

    print("Callback list")
    callback_list, log_path = self.callbackList("genderlogs")


    history = self.model.fit(X_train, y_train, batch_size=batch_size , validation_batch_size=batch_size,
                           validation_data=(X_test, y_test),epochs=60,
                           callbacks=callback_list,shuffle=True)
    prediction = self.model.predict(X_test)

    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)

    prediction = (prediction > 0.5).astype(int)

    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)

    acc = accuracy_score(y_test, prediction)
    prec, recall, f1, support = precision_recall_fscore_support(y_test, prediction, average='macro')
    confusionMatrix = confusion_matrix(y_test, prediction)

    print(f"Accuracy is {acc}")
    print(f"Precision is:  {prec}")
    print(f"Recall is:  {recall}")
    print(f"F1 is :  {f1}")
    print(confusionMatrix)
    return history


  def regression_training(self, batch_size=80):
    X_train, X_test, y_train, y_test = self.regression_inputs()
    print("Callback list")
    callback_list, log_path = self.callbackList("agelogs")
    print("Compile")

    print("Fit")
    history = self.model.fit(X_train, y_train, batch_size=batch_size, validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=60,
                            callbacks=callback_list,shuffle=True)


    prediction = self.model.predict(X_test)
    rSquared = r2_score(y_test, prediction.flatten())
    meanAbsoluteError = mean_absolute_error(y_test, prediction.flatten())
    meanSquaredError = mean_squared_error(y_test, prediction.flatten())
    rootMeanSquaredError = mean_squared_error(y_test, prediction.flatten(), squared=False)
    print(f"R-Squared value is {rSquared}")
    print(f"Mean absolute error is {meanAbsoluteError}")
    print(f"Mean squared error is {meanSquaredError}")
    print(f"Root mean squared error is {rootMeanSquaredError}")
    return history


  def age_training(self, batch_size=80):
    X_train, X_test, y_train, y_test = self.age_inputs()
    print("Callback list")
    callback_list, log_path = self.callbackList("agelogs")
    print("Compile")

    print("Fit")
    history = self.model.fit(X_train, y_train, batch_size=batch_size, validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=60,
                            callbacks=callback_list,shuffle=True)

    prediction = self.model.predict(X_test)
    print("prediction:", prediction.shape)
    print("True val:", y_test.shape)
    prediction= np.argmax(prediction, axis=1)
    acc = accuracy_score(y_test, prediction)
    prec, recall, f1, support = precision_recall_fscore_support(y_test, prediction, average='macro')
    confusionMatrix = confusion_matrix(y_test, prediction)
    print(f"Accuracy is {acc}")
    print(f"Precision is:  {prec}")
    print(f"Recall is:  {recall}")
    print(f"F1 is :  {f1}")
    print(confusionMatrix)
    return history

# Model for building the specific versions of DenseNet
@keras.utils.register_keras_serializable(package='Custom', name='DenseNet')
class DENSENET_GENDER(DenseNet):
  def __init__(self, *args, **kwargs):
    super(DENSENET_GENDER, self).__init__(*args, **kwargs)
    outputs = Dense(1, activation='sigmoid')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='DenseNet')
class DENSENET_AGE(DenseNet):
  def __init__(self, *args, **kwargs):
    super(DENSENET_AGE, self).__init__(*args, **kwargs)
    outputs = Dense(7, activation='softmax')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

@keras.utils.register_keras_serializable(package='Custom', name='DenseNet')
class DENSENET_REGRESSION(DenseNet):
  def __init__(self, *args, **kwargs):
    super(DENSENET_REGRESSION, self).__init__(*args, **kwargs)
    outputs = Dense(1, activation='linear')(self.baseLayers)
    model = Model(self.inputs, outputs)
    self.model = model
    self.model.compile(optimizer="adam", loss="mean_squared_error", metrics=['mae'])


model = DENSENET_REGRESSION()
history = model.regression_training()

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/ValidationData/2_1_0_20170109191147581.jpg.chip.jpg
/content/drive/My Drive/ValidationData/7_0_2_20161219190141683.jpg.chip.jpg
/content/drive/My Drive/ValidationData/52_1_1_20170113000523249.jpg.chip.jpg
/content/drive/My Drive/ValidationData/11_1_0_20170109204833458.jpg.chip.jpg
/content/drive/My Drive/ValidationData/49_0_0_20170119202355566.jpg.chip.jpg
/content/drive/My Drive/ValidationData/24_1_2_20170116171158821.jpg.chip.jpg
/content/drive/My Drive/ValidationData/41_1_1_20170117171024861.jpg.chip.jpg
/content/drive/My Drive/ValidationData/51_0_3_20170119200305083.jpg.chip.jpg
/content/drive/My Drive/ValidationData/26_0_1_20170116024841535.jpg.chip.jpg
/content/drive/My Drive/ValidationData/24_1_1_20170112234345639.jpg.chip.jpg
/content/drive/My Drive/ValidationData/29_1_0_20170103180229287.jpg.chip.jpg
/content/drive/My Drive/ValidationData/18_1_0_20170109214608184.jpg.chip.jpg
/content/drive/My Drive/Val

# Transfer learning

In [ ]:
from keras.applications import VGG16, VGG19, ResNet50, DenseNet121
from keras import models



class VGG19_TransferLearning(Model_Base):
    def __init__(self, *args, **kwargs):
        super(VGG19_TransferLearning, self).__init__(*args, **kwargs)
        base = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        self.add(base)
        self.add(Flatten())

class VGG19_TransferLearning_AGE(VGG19_TransferLearning):
  def __init__(self, *args, **kwargs):
    super(VGG19_TransferLearning_AGE, self).__init__(*args, **kwargs)
    self.add(Dense(7, activation='softmax'))
    self.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model = VGG19_TransferLearning_AGE()
history = model.regression_training()

In [ ]:
from keras.applications import VGG16, VGG19, ResNet50, DenseNet121
from keras import models

class transferLearningModel(Sequential):
  def __init__(self, ModelType, InputType, Regression = False):
    super().__init__()

    classNum = MLDataUtils.get_class_num(InputType)
    print(str(classNum))

    if ModelType == "VGG16":
      base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif ModelType == "VGG19":
      base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif ModelType == "ResNet50":
      base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif ModelType == "DenseNet121":
      base = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    else:
      raise ValueError("Value Error")

    #for layer in base.layers:
      #layer.trainable = False

    self.add(base)
    self.add(Flatten())

    if Regression:
      self.add(Dense(1, activation='linear'))
      self.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    else:
      self.add(Dense(classNum, activation='softmax'))
      self.compile(optimizer= "adam",
                 loss="categorical_crossentropy",
                 metrics=['accuracy'])

  def Gender_training(self, batch_size=16):
    print("Input")
    self.summary()
    X_train, X_test, y_train, y_test = MLDataUtils.set_inputs("gender")

    print("Callback list")
    callback_list, log_path = MLDataUtils.callbackList("genderlogs")

    print("Fit")
    history = self.fit(X_train, y_train, batch_size=batch_size , validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=60,
                            callbacks=callback_list,shuffle=True)
    prediction = self.predict(X_test)
    acc = MLDataUtils.classification_eval_metrics(prediction, y_test)
    MLDataUtils.update_h5_name(log_path, history, acc)
    return history

  def Age_training(self, batch_size=10, Regression=False):
    print("Input")
    self.summary()
    X_train, X_test, y_train, y_test = MLDataUtils.set_inputs("age", Regression)

    print("Callback list")
    callback_list, log_path = MLDataUtils.callbackList("agelogs")

    print("Fit")
    history = self.fit(X_train, y_train, batch_size=batch_size , validation_batch_size=batch_size,
                            validation_data=(X_test, y_test),epochs=60,
                            callbacks=callback_list,shuffle=True)
    prediction = self.predict(X_test)
    if Regression:
      meanAbsoluteError = MLDataUtils.regression_eval_metrics(prediction, y_test)
      MLDataUtils.update_h5_name(log_path, history, meanAbsoluteError)
    return history
"""
Following is the two methods which set up the model class and call the correct training methods
"""
def selectModelType(arch):
  typeSelect = input("Please select a model type [(1 = Gender)(2 = Age)(3=Regression)]")
  reg = False
  if typeSelect == "1":
    print("Gender")
    model = transferLearningModel(arch, "gender")
  elif typeSelect == "2":
    print("Age")
    model = transferLearningModel(arch, "age")
    reg = False
  elif typeSelect == "3":
    print("Age regression")
    model = transferLearningModel(arch, "age", Regression=True)
    reg = True
  else:
    raise ValueError("Input must be either[1,2,3]")

  print("Choosing Model")
  if typeSelect == "1":
    history = model.Gender_training()
  else:
    history = model.Age_training(Regression=reg)

def selectModelArchitecture():
  modelArchitecture = input("Please select a model architecture [(1 = VGG16)(2 = VGG19)(3=ResNet50)(4=DenseNet121)]")

  if modelArchitecture == "1":
    print("VGG16")
    inputType = "VGG16"
  elif modelArchitecture == "2":
    print("VGG19")
    inputType = "VGG19"
  elif modelArchitecture == "3":
    print("ResNet50")
    inputType = "ResNet50"
  elif modelArchitecture == "4":
    print("DenseNet121")
    inputType = "DenseNet121"
  else:
    raise ValueError("Input must be either[1,2,3,4]")

  selectModelType(inputType)


selectModelArchitecture()

Please select a model architecture [(1 = VGG16)(2 = VGG19)(3=ResNet50)(4=DenseNet121)]1
VGG16
Please select a model type [(1 = Gender)(2 = Age)(3=Regression)]1
Gender
2
Choosing Model
Input
Model: "transfer_learning_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_1 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 14764866 (56.32 MB)
Trainable params: 14764866 (56.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Setting gender target
['/content/drive/My Drive/GreyscaleData_test/36_1_1

KeyboardInterrupt: 

In [ ]:
from keras.applications import VGG16, VGG19, ResNet50, DenseNet121
from keras import models

class transferLearningModel(Model_Base):
  def __init__(self, ModelType, InputType, *args, **kwargs):
    super(transferLearningModel, self).__init__(*args, **kwargs)


    if ModelType == "VGG16":
      base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif ModelType == "VGG19":
      base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif ModelType == "ResNet50":
      base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif ModelType == "DenseNet121":
      base = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    else:
      raise ValueError("Value Error")

    #for layer in base.layers:
      #layer.trainable = False

    self.add(base)
    self.add(Flatten())

    if InputType == "regression":
      self.add(Dense(1, activation='linear'))
      self.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    elif InputType == "age":
      self.add(Dense(7, activation='softmax'))
      self.compile(optimizer= "adam",
                 loss="sparse_categorical_crossentropy",
                 metrics=['accuracy'])
    elif InputType == "gender":
      self.add(Dense(1, activation='sigmoid'))
      self.compile(optimizer= "adam",
                 loss="binary_crossentropy",
                 metrics=['accuracy'])


"""
Following is the two methods which set up the model class and call the correct training methods
"""
def selectModelType(arch):
  typeSelect = input("Please select a model type [(1 = Gender)(2 = Age)(3=Regression)]")
  reg = False
  if typeSelect == "1":
    print("Gender")
    model = transferLearningModel(arch, "gender")
    history = model.gender_training()
  elif typeSelect == "2":
    print("Age")
    model = transferLearningModel(arch, "age")
    history = model.age_training()
  elif typeSelect == "3":
    print("Age regression")
    model = transferLearningModel(arch, "regression")
    history = model.gender_training()
  else:
    raise ValueError("Input must be either[1,2,3]")



def selectModelArchitecture():
  modelArchitecture = input("Please select a model architecture [(1 = VGG16)(2 = VGG19)(3=ResNet50)(4=DenseNet121)]")

  if modelArchitecture == "1":
    print("VGG16")
    inputType = "VGG16"
  elif modelArchitecture == "2":
    print("VGG19")
    inputType = "VGG19"
  elif modelArchitecture == "3":
    print("ResNet50")
    inputType = "ResNet50"
  elif modelArchitecture == "4":
    print("DenseNet121")
    inputType = "DenseNet121"
  else:
    raise ValueError("Input must be either[1,2,3,4]")

  selectModelType(inputType)


selectModelArchitecture()

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/ValidationData/22_1_0_20170116220950381.jpg.chip.jpg
/content/drive/My Drive/ValidationData/4_0_3_20161220223111195.jpg.chip.jpg
/content/drive/My Drive/ValidationData/32_1_0_20170117182452621.jpg.chip.jpg
/content/drive/My Drive/ValidationData/35_0_0_20170117180552033.jpg.chip.jpg
/content/drive/My Drive/ValidationData/50_1_0_20170113210319697.jpg.chip.jpg
/content/drive/My Drive/ValidationData/36_1_4_20170112233252616.jpg.chip.jpg
/content/drive/My Drive/ValidationData/61_1_0_20170109150858084.jpg.chip.jpg
/content/drive/My Drive/ValidationData/22_1_3_20170104222220913.jpg.chip.jpg
/content/drive/My Drive/ValidationData/62_1_0_20170110175833643.jpg.chip.jpg
/content/drive/My Drive/ValidationData/2_1_3_20161219225435504.jpg.chip.jpg
/content/drive/My Drive/ValidationData/1_1_4_20161221201827921.jpg.chip.jpg
/content/drive/My Drive/ValidationData/10_1_0_20170109203427416.jpg.chip.jpg
/content/drive/My Drive/Vali